# First PROTOTYPE

In [10]:
import os
from dotenv import  load_dotenv
from openai  import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import json
from pydub import AudioSegment
from pydub.playback import play
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

openai = OpenAI()
claude = anthropic.Anthropic()

In [3]:
gpt_system = "You are a helpful AI assistant working for a travel agency. \
You assist customers with booking tickets, checking prices, and providing travel information. \
You should provide accurate, up-to-date details about flights, hotels, and transportation options. \
Your goal is to help the customer plan their trip smoothly. The conversation will be done in 8 back-and-forth exchanges."

claude_system = "You are a curious customer who wants to travel and gather information. \
You are looking for ticket prices, available flights, and recommendations for your trip. \
You ask detailed questions about different destinations, travel options, and costs. \
Your goal is to get the best travel advice possible. The conversation will be done in 8 back-and-forth exchanges."

gpt_messages = ["Hello!"]
claude_messages = ["Hi!"]

In [4]:
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499",
    "new_york": "$650",
    "los_angeles": "$750",
    "sydney": "$1600",
    "dubai": "$1200",
    "rome": "$850",
    "singapore": "$1300",
    "toronto": "$700",
    "bangkok": "$1100",
    "madrid": "$780",
    "hong_kong": "$1250",
    "amsterdam": "$820",
    "istanbul": "$950",
    "seoul": "$1350",
    "cairo": "$880",
    "rio_de_janeiro": "$980",
    "cape_town": "$1450",
    "beijing": "$1380",
    "mexico_city": "$900",
    "buenos_aires": "$1050",
    "moscow": "$1200",
    "vienna": "$830",
    "lisbon": "$810",
    "são_paulo": "$1020",
    "prague": "$790",
    "stockholm": "$860",
    "helsinki": "$870",
    "oslo": "$890",
    "budapest": "$740",
    "warsaw": "$720",
    "brussels": "$805",
    "zurich": "$880",
    "geneva": "$895",
    "athens": "$910",
    "jakarta": "$1250",
    "manila": "$1220",
    "kuala_lumpur": "$1180",
    "taipei": "$1280",
    "ho_chi_minh_city": "$1120",
    "hanoi": "$1110",
    "mumbai": "$1150",
    "delhi": "$1170",
    "bangalore": "$1190",
    "chennai": "$1160",
    "santiago": "$1080",
    "bogota": "$980",
    "lima": "$970",
    "auckland": "$1650",
    "nairobi": "$1320",
    "lagos": "$1400",
    "doha": "$1250",
    "riyadh": "$1275"
}

In [6]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
tools = [{"type": "function", "function": price_function}]

In [8]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [9]:
def artist(city):
    image_response = openai.images.generate(
        model='dall-e-3',
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:


def talker_claude(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

def talker_gpt(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)
    
  

In [ ]:


# def call_gpt(message,history):
#     messages = [{"role": "system", "content": system_message}] + history +  [{"role":"user", "content":message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     image = None

#     if response.choices[0].finish_reason == 'tool_calls':
#         message  = response.choices[0].message
#         response,city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         image = artist(city)

#         response = openai.chat.completions.create(model=MODEL, messages=messages)
#     reply = response.choices[0].message.content
#     history += [{"role":"assistant", "content":reply}]
#     talker_claude(reply)

#     return history,image,""


In [ ]:
def call_claude(history,message):
    messages = [{"role": "system", "content": claude_system}] + history +  [{"role":"user", "content":message}]
    
    completion = claude.messages.create(
        model = 'claude-3-haiku-20240307',
        messages=messages,
        system=claude_system,
        max_tokens=500,
    )

    reply = completion.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    talker_claude(reply)
    return history,reply

In [ ]:
# def chat(message,history):
#     messages = [{"role": "system", "content": system_message}] + history +  [{"role":"user", "content":message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     image = None

#     if response.choices[0].finish_reason == 'tool_calls':
#         message  = response.choices[0].message
#         response,city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         image = artist(city)

#         response = openai.chat.completions.create(model=MODEL, messages=messages)
#     reply = response.choices[0].message.content
#     history += [{"role":"assistant", "content":reply}]
#     talker_claude(reply)

#     return history,image,""


In [ ]:
	
# gpt_messages = ["Hi there"]
# claude_messages = ["Hi"]

# print(f"GPT:\n{gpt_messages[0]}\n")
# print(f"Claude:\n{gpt_messages[0]}\n")


# for i in range(5):
#     gpt_next = call_gpt()
#     print(f"GPT:\n{gpt_next}\n")
#     gpt_messages.append(gpt_next)


#     claude_next = call_claude()
#     print(f"Claude:\n{claude_next}\n")
#     claude_messages.append(claude_next)

In [ ]:
# with gr.Blocks() as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500 , type='messages') #gpt
#         image_output = gr.Image(height=500)

#     with gr.Row():
#         entry = gr.Textbox(label = 'chat with AI') #claude
#         with gr.Row():
#             clear = gr.Button('clear')
#         entry.submit(call_gpt, [entry, chatbot], [chatbot, image_output,entry]).then(
#                     call_claude, [chatbot, ]
#         )
#         clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)
# ui.launch()


In [ ]:
def call_claude(history, message):
    messages = [{"role": "system", "content": claude_system}] + history + [{"role": "user", "content": message}]
    
    completion = claude.messages.create(
        model='claude-3-haiku-20240307',
        messages=messages,
        system=claude_system,
        max_tokens=500,
    )

    reply = completion.choices[0].message.content
    history.append({"role": "assistant", "content": reply})
    
    talker_claude(reply)
    return history, reply  # Update chatbot & entry with Claude's reply


def call_gpt(message, history):
    messages = [{"role": "system", "content": gpt_system}] + history + [{"role": "user", "content": message}]
    
    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append({"role": "assistant", "content": response})
        image = artist(city)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history.append({"role": "assistant", "content": reply})
    
    talker_gpt(reply)
    return history, image, reply  # Update chatbot, image, and pass reply to Claude

In [ ]:
def call_claude(history):
    # Prepare messages - exclude system role from messages list
    messages = []
    for msg in history:
        # Convert "system" messages to "user" messages for Claude
        role = "user" if msg["role"] == "system" else msg["role"]
        messages.append({"role": role, "content": msg["content"]})
    
    completion = claude.messages.create(
        model='claude-3-haiku-20240307',
        messages=messages,
        system=claude_system,  # System message passed separately
        max_tokens=500,
    )
    
    reply = completion.content[0].text
    updated_history = list(history) + [{"role": "assistant", "content": reply}]
    
    talker_claude(reply)  # Your voice function
    return updated_history, reply


def call_gpt(history):
    # Prepare messages - include system message at start if history is empty
    messages = [{"role": "system", "content": gpt_system}] if not history else []
    messages += list(history)
    
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        tools=tools
    )
    
    reply = response.choices[0].message.content
    image = None
    
    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        reply, city = handle_tool_call(tool_call)
        image = artist(city)
    
    updated_history = list(history) + [{"role": "assistant", "content": reply}]
    
    talker_gpt(reply)  # Your voice function
    return updated_history, image, reply


# with gr.Blocks() as ui:
#     history = gr.State([])
    
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500)
#         image_output = gr.Image(height=500)
    
#     start_btn = gr.Button("Start Conversation")
    
#     def start_conversation(history):
#         # Initialize with Claude's system message as first user message
#         history = [{"role": "user", "content": claude_system}]
        
#         # Claude speaks first
#         history, claude_reply = call_claude(history)
#         chat_history = [(claude_reply, "")]
        
#         # GPT responds
#         history, image, gpt_reply = call_gpt(history)
#         chat_history[-1] = (claude_reply, gpt_reply)
        
#         return history, chat_history, image
    
#     start_btn.click(
#         start_conversation,
#         inputs=[history],
#         outputs=[history, chatbot, image_output]
#     )

# ui.launch()